# Aries Basic Controller - Alice

## This notebook works through the present proof protocol from the Verifier's perspective, it should be run alongside the prover notebook from Bob's perspective. 

## Before running through these two notebooks you should run through the Issuer/Holder flow found in these two notebooks - [issuer](http://localhost:8888/notebooks/issuer.ipynb) [holder](http://localhost:8889/notebooks/holder.ipynb).

If unfamiliar with the present-proof protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0037-present-proof)


## 1. Instatiate Controller for Alice's Agent

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
alice_agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic present_proof, printing out the payload. The agent calls to this webhook every time it receives a present proof protocol message from another agent. 

The code for this protocol can be found [here](https://github.com/hyperledger/aries-cloudagent-python/tree/master/aries_cloudagent/protocols/issue_credential).

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(alice_agent_controller.listen_webhooks())

def proof_handler(payload):
    print("Handle present proof")
    print(payload)

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
alice_agent_controller.register_listeners([proof_listener], defaults=True)


## 3. Check the agent has an active connection

**Note: An active connection is required, this should have been established on start up through the python script create_connection.py in the setup folder. If not it is possible to run through the did-exchange tutorial to create one between Alice and Bob**

* [Alice](http://localhost:8888/notebooks/did-exchange-inviter.ipynb)
* [Bob](http://localhost:8889/notebooks/did-exchange-invitee.ipynb)

In [3]:
try:
    response = await alice_agent_controller.connections.get_connections()
    results = response['results']
    print("Results : ", results)
    print('\n')
    if len(results) > 0:
        connection = response['results'][0]
        print("Connection :", connection)
        if connection['state'] == 'active':
            connection_id = connection["connection_id"]
            print("\nActive Connection ID : ", connection_id)
        else:
            print("\nNo active connection found - wait a bit and execute again")
    else:
        print("You must create a connection")
except ConnectionRefusedError as e:
    print(repr(e))

Results :  [{'my_did': '6fgDnPSBh5gXgupAqGAH4h', 'their_label': 'Bob', 'initiator': 'external', 'created_at': '2020-09-03 11:22:06.585012Z', 'invitation_key': 'H1AHmexKtYx4bZbd257NCGdHcYXQsRmY419zLvT28Adw', 'routing_state': 'none', 'state': 'active', 'updated_at': '2020-09-03 11:22:18.709593Z', 'accept': 'manual', 'invitation_mode': 'once', 'request_id': '4be6a88c-3dec-4f3b-aac8-724e4369f7a7', 'connection_id': '4a3bfb5d-483c-4ba3-b7f8-3755d09a3fb9', 'their_did': 'L7arMaEZQCbJq1eTCmL1pu'}]


Connection : {'my_did': '6fgDnPSBh5gXgupAqGAH4h', 'their_label': 'Bob', 'initiator': 'external', 'created_at': '2020-09-03 11:22:06.585012Z', 'invitation_key': 'H1AHmexKtYx4bZbd257NCGdHcYXQsRmY419zLvT28Adw', 'routing_state': 'none', 'state': 'active', 'updated_at': '2020-09-03 11:22:18.709593Z', 'accept': 'manual', 'invitation_mode': 'once', 'request_id': '4be6a88c-3dec-4f3b-aac8-724e4369f7a7', 'connection_id': '4a3bfb5d-483c-4ba3-b7f8-3755d09a3fb9', 'their_did': 'L7arMaEZQCbJq1eTCmL1pu'}

Active Co

## 4. Continue in [prover](http://localhost:8889/notebooks/prover.ipynb) Notebook

## 7. Send Proof Request

Request a proof of Opemined community member to prove community membership.



### 7.1. First we build the request object, a json. In the future we aim to develop a builder class that abstracts this away.

### TODO add better documentation to this whole process. Design for someome who has no idea what a proof request is. Hopefully the builder class will make that easier?

### Easy step for now would be to split up the sections and document each bit. Like the req_attrs. and req_preds. What restrictions are available etx.

In [6]:
response = await alice_agent_controller.wallet.get_public_did()
print(response)
issuer_did = response["result"]["did"]

print("Request proof of Name and Age range from Bob")
#Set some variables

revocation = False
SELF_ATTESTED = True
exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "name", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "skill", "restrictions": [{"issuer_did": issuer_did}]},
]

if revocation:
    req_attrs.append(
        {
            "name": "skill",
            "restrictions": [{"issuer_did": agent.did}],
            "non_revoked": {"to": int(time.time() - 1)},
        },
    )

if SELF_ATTESTED:
    # test self-attested claims
    req_attrs.append({"name": "country"},)

#Set predicates for Zero Knowledge Proofs
req_preds = [
    # test zero-knowledge proofs
    {
        "name": "age",
        "p_type": ">=",
        "p_value": 21,
        "restrictions": [{"issuer_did": issuer_did}],
    }
]

indy_proof_request = {
    "name": "Proof of Personal Information",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}

if revocation:
    indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

{'result': {'did': 'PQRXDxdGqQGSZ8z69p4xZP', 'verkey': 'DDEKJtBjzaXAJtpwetdPiXH5s4rNUEzG18V15QoLRcZZ', 'public': 'true'}}
Request proof of Name and Age range from Bob


### 7.2. Send the proof request to Bob

Bob is identified through the connection_id

In [7]:
response = await alice_agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)


{'thread_id': 'd0dec549-b119-4f05-9384-1edfcd634bc5', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_name_uuid': {'name': 'name', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_skill_uuid': {'name': 'skill', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_country_uuid': {'name': 'country'}}, 'requested_predicates': {'0_age_GE_uuid': {'name': 'age', 'p_type': '>=', 'p_value': 21, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '1012453674509974792060177'}, 'auto_present': False, 'initiator': 'self', 'created_at': '2020-09-03 11:59:11.247203Z', 'state': 'request_sent', 'updated_at': '2020-09-03 11:59:11.247203Z', 'presentation_exchange_id': '9b98163e-8973-41cc-963e-1c1454ba9b82', 'trace': False, 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': 'd0dec549-b119-4f05-9384-1edfcd634bc5', 'request_pre

## 8. Continue in the prover notebook

## 11. Verify Proof Presentation

This is checking the signatures on the credentials presented against the credential schema and definition id stored and resolvable on the ledger.

### TODO: It is not clear to me how we actually take this big blob of data and parse it to make a meaningful, true false. Yes this proof is valid, no it is not. I think this is perhaps a event that gets fired to the handler? Additionally it would be good to show how you transform this to get the attributes that were proven. Should the controller provide some helper functions for this? Not sure, think it is more the role of the business logic using the controller. But would still be useful to show in the notebooks.

In [11]:
verify = await alice_agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify)


ClientResponseError: 400, message='Presentation exchange 9b98163e-8973-41cc-963e-1c1454ba9b82 in verified state (must be presentation_received)', url=URL('http://alice-agent:8021/present-proof/records/9b98163e-8973-41cc-963e-1c1454ba9b82/verify-presentation')

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/helpers/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/helpers/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/helpers/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/

## Not all the proofs api has been covered in this example.

### TODO: At least list API calls not documented.

## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [26]:
response = await alice_agent_controller.terminate()
print(response)

None


NameError: name 'bob_agent_controller' is not defined